In [ ]:
###### @Mike 
# Can you add a few sentences about how you got the data

In [1]:
# Import necessary modules
import os
import glob

import pandas as pd
import statsmodels.formula.api as smf

import plotly.graph_objects as go # for interactive plot with human defined labels
import plotly.express as px

In [21]:
# Load necessary data
path = "/Users/ZXue/src/speeding-up-sci-correlation/"
df = pd.read_table(os.path.join(path, "TARA_025-merged-only-those-in-both-full-TPM.tsv"))
df.head() # a look at the sample



,Gene,DNA,RNA
0,TOBG-MED-1076_1101,3.578633,12.992625
1,TOBG-MED-1076_1116,0.714860,4.037256
2,TOBG-MED-1076_1131,7.727037,5.454919
3,TOBG-MED-1076_1151,2.859440,15.572275
4,TOBG-MED-1076_1195,8.813051,12.379700


In [22]:
## Add KO ID to df
df_anno = pd.read_table(os.path.join(path, "Our-Kegg-annotations.tsv"))
df_anno.head()

df = df.merge(df_anno, on='Gene')
# remove genes that are not annotated with a KO ID
df.dropna()

,Gene,DNA,RNA,KO_ID
0,TOBG-MED-1076_1101,3.578633,12.992625,K01940
1,TOBG-MED-1076_1116,0.714860,4.037256,K00962
2,TOBG-MED-1076_1131,7.727037,5.454919,K01696
3,TOBG-MED-1076_1151,2.859440,15.572275,K02990
6,TOBG-MED-1076_1298,2.923141,4.295652,K00347
...,...,...,...,...
24039,TOBG-MED-831_959,1.353264,7.915675,K00005
24040,TOBG-MED-831_960,8.061755,34.266264,K03696
24041,TOBG-MED-831_978,0.950462,1.150253,K03455
24042,TOBG-MED-831_98,0.479963,3.630331,K08309


In [23]:
## Add pathway info to df
df_path = pd.read_table(os.path.join(path, "Kegg-Orthology-table.tsv"))
df_path.head()

df = df.merge(df_path, on='KO_ID')
df.head()

,Gene,DNA,RNA,KO_ID,Category1,Category2,Category3,description
0,TOBG-MED-1076_1101,3.578633,12.992625,K01940,Metabolism,Overview,01230 Biosynthesis of amino acids [PATH:ko01230],"argG, ASS1; argininosuccinate synthase [EC:6...."
1,TOBG-MED-1076_1101,3.578633,12.992625,K01940,Metabolism,Amino acid metabolism,"00250 Alanine, aspartate and glutamate metabol...","argG, ASS1; argininosuccinate synthase [EC:6...."
2,TOBG-MED-1076_1101,3.578633,12.992625,K01940,Metabolism,Amino acid metabolism,00220 Arginine biosynthesis [PATH:ko00220],"argG, ASS1; argininosuccinate synthase [EC:6...."
3,TOBG-MED-1076_1101,3.578633,12.992625,K01940,Human Diseases,Cardiovascular diseases,05418 Fluid shear stress and atherosclerosis [...,"argG, ASS1; argininosuccinate synthase [EC:6...."
4,TOBG-MED-590_1583,23.569356,25.271020,K01940,Metabolism,Overview,01230 Biosynthesis of amino acids [PATH:ko01230],"argG, ASS1; argininosuccinate synthase [EC:6...."


In [24]:
# Make the figure with scatter dots
tracedot = go.Scatter(x=df['DNA'], # User should change this based on dataframe columns
                      y=df['RNA'], # User defined 
                      mode='markers', # for scatter plot
                      text=df['KO_ID'], # hover text
                      #color=df['Category1'],
                      name='Data') # User defined


In [2]:
# Initialise and fit linear regression model using `statsmodels`
### tutorial https://towardsdatascience.com/introduction-to-linear-regression-in-python-c12a072bedf0
model = smf.ols('RNA ~ DNA', data=df)
model = model.fit()
# get linear regression parameters
model.params


NameError: name 'smf' is not defined

In [7]:
# get linear regression R^2 
model.rsquared

0.02598684167801979

In [8]:
# get linear regression p-value
model.pvalues

Intercept     7.292660e-07
DNA          8.397350e-103
dtype: float64

In [9]:
# linear regression equation
max_val = df['DNA'].max()
df2 = [[0, model.params[0]],[max_val, model.params[1]*max_val+model.params[0]]]
df2 = pd.DataFrame(df2, columns = ['xi', 'y']) 
 

# Make the line graph 
trace_ln = go.Scatter(
                  x=df2['xi'],
                  y=df2['y'],
                  mode='lines',
                  name='Fit'
                  )


In [1]:
# Make the text string indicating the linear regression model

eq_txt = "R\N{SUPERSCRIPT TWO} = {:.3f}, \y = {:.3f}x+{:.3f}".format(model.rsquared, model.params[1], model.params[0])

annotation = go.layout.Annotation(
                  x=df['DNA'].max(),
                  y=df['RNA'].max(),
                  text=eq_txt,
                  showarrow=False,
                  font=go.Font(size=16)
                  )
layout = go.Layout(annotations=[annotation])


# Tie everything together to a figure
fig = go.Figure([tracedot,trace_ln], layout=layout)


# Edit the figure layout 
fig.update_layout(title='Gene correlations between metagenomics and metatranscriptomic resutls',
                  xaxis_title='Count (Metagenomics)',
                  yaxis_title='Count (Metatranscriptomics)')

fig.show()

NameError: name 'model' is not defined

In [ ]:
### go trace plotting can not set color to dots by category 
## so I am now trying to set color with px plotting
## and add line and text through adding trace
figdot = px.scatter(df, x="DNA", y="RNA", color="Category2", hover_data=['KO_ID'])

# a "dummy" scatter plot trace that is used to carry the text 
trace_txt = go.Scatter(   
    x=[df['DNA'].mean()], ##########
    y=[df['RNA'].max()],
    mode='markers+text',
    text=eq_txt,
    marker=dict(size = 1),
    name='linear regression model',
    textposition='bottom center'
)

# add a trace to represent 1:1 regression (y=x)
df3 = [[0, 0],[max_val, max_val]]
df3 = pd.DataFrame(df3, columns = ['xi', 'y']) 
trace_yx = go.Scatter(
                  x=df3['xi'],
                  y=df3['y'],
                  mode='lines',
                  line = dict(color='grey', width=4, dash='dot'),
                  name='1:1 line'
                  )


figdot.add_trace(trace_ln).add_trace(trace_txt).add_trace(trace_yx)
#go.Figure(figdot, layout=layout) # why is the text in layout settings not showing up?!
figdot.show()



In [49]:
'''An example of interesting results: K01574 was expressed at very high level in metatrancriptomic results but its "copy number"
is relatively low by metagenomic measurements. '''

'''This KO ID is linked to both carbohydrate and lipid metabolism, suggesting robust microbial activities
within in the sampled Tara oceam specimen.'''

df[df['KO_ID']  == 'K02040']





,Gene,DNA,RNA,KO_ID,Category1,Category2,Category3,description
9592,TOBG-MED-587_690,11.446196,37.039658,K02040,Environmental Information Processing,Membrane transport,02010 ABC transporters [PATH:ko02010],pstS; phosphate transport system substrate-bi...
9593,TOBG-MED-587_690,11.446196,37.039658,K02040,Environmental Information Processing,Signal transduction,02020 Two-component system [PATH:ko02020],pstS; phosphate transport system substrate-bi...
9594,TOBG-MED-587_690,11.446196,37.039658,K02040,Human Diseases,Infectious diseases,05152 Tuberculosis [PATH:ko05152],pstS; phosphate transport system substrate-bi...
9595,TOBG-MED-591_1040,94.927042,7.677695,K02040,Environmental Information Processing,Membrane transport,02010 ABC transporters [PATH:ko02010],pstS; phosphate transport system substrate-bi...
9596,TOBG-MED-591_1040,94.927042,7.677695,K02040,Environmental Information Processing,Signal transduction,02020 Two-component system [PATH:ko02020],pstS; phosphate transport system substrate-bi...
...,...,...,...,...,...,...,...,...
9662,TOBG-MED-831_13,0.502104,11.241493,K02040,Environmental Information Processing,Signal transduction,02020 Two-component system [PATH:ko02020],pstS; phosphate transport system substrate-bi...
9663,TOBG-MED-831_13,0.502104,11.241493,K02040,Human Diseases,Infectious diseases,05152 Tuberculosis [PATH:ko05152],pstS; phosphate transport system substrate-bi...
9664,TOBG-MED-831_64,28.375790,217.281645,K02040,Environmental Information Processing,Membrane transport,02010 ABC transporters [PATH:ko02010],pstS; phosphate transport system substrate-bi...
9665,TOBG-MED-831_64,28.375790,217.281645,K02040,Environmental Information Processing,Signal transduction,02020 Two-component system [PATH:ko02020],pstS; phosphate transport system substrate-bi...


In [42]:
## To make a web-based application using plotly?


42.77276000869718